In [31]:
import cv2
import numpy as np
import csv
from PIL import Image

def merge_images(img1, img2):
    cloud_pixel = [255, 255, 255]
    merged = np.where(img1 == cloud_pixel, img2, img1)
    return merged

def get_centroid(contour):
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
    else:
        cx, cy = 0, 0
    return cx, cy

def extract_positions_and_draw_circles(img, img_path):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

    # Find all contours in the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    min_area = 0.001 * img.shape[0] * img.shape[1]
    filtered_contours = [contour for contour in contours if cv2.contourArea(contour) > min_area]
    sorted_contours = sorted(filtered_contours, key=cv2.contourArea, reverse=True)

    if len(sorted_contours) < 6:
        print(f"Unexpected number of contours: {len(sorted_contours)}")
        return []

    animal_contours = sorted_contours[1:6]
    positions = []
    for contour in animal_contours:
        cx, cy = get_centroid(contour)
        positions.append((cx, cy))
        cv2.circle(img, (cx, cy), 20, (0, 255, 0), 2)
    modified_img_path = img_path.replace("_sample0.png", "_marked.png")
    cv2.imwrite(modified_img_path, img)

    return positions

predictions = []

for i in range(1000):
    img_path1 = f"level_05/test_data/field{i:03d}_sample0.png"
    img_path2 = f"level_05/test_data/field{i:03d}_sample1.png"
    img1 = np.asarray(Image.open(img_path1).convert('RGB'))
    img2 = np.asarray(Image.open(img_path2).convert('RGB'))
    merged_img = merge_images(img1, img2)
    positions = extract_positions_and_draw_circles(merged_img, img_path1)

    flattened_positions = [coord for position in positions for coord in position]
    predictions.append(flattened_positions)

with open('predictions.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(predictions)
